<b><u>Basic Introduction to installing packages using Jupyter and reading/writing tables in GCP</u></b>

How to install packages in jupyter notebook?
- write using the statement <b>!pip install <package_name></b> as shown below

In [ ]:
!pip install google-cloud-bigquery

Reading a csv file as a dataframe

In [1]:
import pandas as pd

# CHANGE NAME OF FILE HERE WHICH YOU WANT TO SEE IN THE DATAFRAME
df = pd.read_csv('C:\\coding\\python\\raw_data\\drivers.csv')

df.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


Loading the above result into the a table in database/dataset

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd


# Getting the GCP Cloud Permissions to connect Jupyter and python to Cloud database
credentials = service_account.Credentials.from_service_account_file('C:\\coding\\python\\credentials\\config.json')
client = bigquery.Client('abhinand-playground',credentials)

# CHANGE NAME OF TABLE HERE AFTER RUNNING CODE TO GET DATAFRAME VALUES
table_id = 'formula1_env.status'

#write_disposition = 'WRITE_APPEND' will append data into the table.
#write_disposition = 'WRITE_TRUNCATE' If the table already exists, BigQuery overwrites the table data.
#write_disposition = 'WRITE_EMPTY' If the table already exists and contains data, a ‘duplicate’ error is returned

job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')


job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 138 rows and 2 columns to formula1_env.status


querying the table so see the result

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct a BigQuery client object.
credentials = service_account.Credentials.from_service_account_file('C:\\coding\\python\\credentials\\config.json')
client = bigquery.Client('abhinand-playground',credentials)

sql = """
select * from formula1_env.status
"""
df_1 = client.query(sql).to_dataframe()
df_1.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


To drop the table - modify the statement below with the table name and run

In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Construct a BigQuery client object.
credentials = service_account.Credentials.from_service_account_file('C:\\coding\\python\\credentials\\config.json')
client = bigquery.Client('abhinand-playground',credentials)

# If the table does not exist, delete_table raises
# google.api_core.exceptions.NotFound unless not_found_ok is True.
table_id = 'formula1_env.status'
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))

Deleted table 'formula1_env.status'.
